In [ ]:
# import necessary libraries and initialize the model to a variable name 'MODEL'
import os 
import sys

MODEL = "llama3.2:1b"

In [ ]:
# Importing the necessary libraries from LangChain and initialize the model 
from langchain_openai.chat_models import ChatOpenAI
from langchain_community.llms import Ollama
from langchain_community.embeddings import OllamaEmbeddings

model = Ollama(model=MODEL)
embeddings = OllamaEmbeddings(model=MODEL)

# Testing the model with a simple prompt 
model.invoke("Tell me a joke about AI")

'A programmer asked an AI, "Will you create a new language that\'s more efficient than English?"\n\nThe AI replied, "I\'m already working on it."'

In [ ]:
# Import the necessary output parser and create a chain to process the model's output (this is optional as it only applies if you want to parse the output in a specific way or format applies when you're getting an output from open ai)
from langchain_core.output_parsers import StrOutputParser

# Create and output parser to handle the models output 
parser = StrOutputParser()

# Create a chain that combines the model and the parser and invoke it with a prompt 
chain = model | parser
chain.invoke("Tell me a brief funny history/n")

'Here\'s a brief, humorous history of the Wi-Fi concept:\n\nThe idea for Wi-Fi was born in the early 1990s at Xerox PARC (Palo Alto Research Center) in California. A team of researchers led by Bob Metcalfe and John O\'Sullivan were trying to find a way to connect devices to each other wirelessly.\n\nAt the time, people used to have to be physically connected to a computer or network using cables, which was clunky and inconvenient. The team realized that with the advent of personal computers becoming more widespread, wireless connectivity would become the norm.\n\nMetcalfe is often credited with coining the term "Wi-Fi" as a combination of the words "wave" and "information," implying that it would be like having a free, internet-based way to share information wirelessly. He also envisioned Wi-Fi being used for more than just networking – he thought it could revolutionize how people worked remotely.\n\nThe first prototype of Wi-Fi was called "ALOHAnet," but it didn\'t quite work as inten

In [ ]:
# Import the necessary libraries, prompt template and create a prompt to format the input for the model 
from langchain.prompts import PromptTemplate

template = """
Answer the question based on the context below. If you can't answer the question, reply "I don't know".

Context: {context}

Question: {question}
"""

prompt = PromptTemplate.from_template(template)
prompt.format(context="Here is some context", question="Here is a question")

'\nAnswer the question based on the context below. If you can\'t answer the question, reply "I don\'t know".\n\nContext: Here is some context\n\nQuestion: Here is a question\n'

In [ ]:
# Import the necessary document loader to load and split the PDF document 
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("2025 Budget Performance.pdf")
pages = loader.load_and_split()
pages

Ignoring wrong pointing object 8 0 (offset 0)
Ignoring wrong pointing object 10 0 (offset 0)
Ignoring wrong pointing object 12 0 (offset 0)
Ignoring wrong pointing object 20 0 (offset 0)
Ignoring wrong pointing object 38 0 (offset 0)


[Document(metadata={'producer': 'macOS Version 10.14.6 (Build 18G9323) Quartz PDFContext', 'creator': 'Word', 'creationdate': "D:20241218144408Z00'00'", 'title': 'Microsoft Word - Document2', 'moddate': "D:20241218144408Z00'00'", 'keywords': '', 'aapl:keywords': '[]', 'source': '2025 Budget Performance.pdf', 'total_pages': 6, 'page': 0, 'page_label': '1'}, page_content="PRESENTATION BY HIS EXCELLENCY , PRESIDENT BOLA AHMED TINUBU, GCFR OF THE 2025 BUDGET OF RESTORATION: SECURING PEACE, REBUILDING PROSPERITY TO A JOINT SESSION OF THE NATIONAL ASSEMBLY ON WEDNESDAY 18TH DECEMBER, 2024   PROTOCOLS: -- The Vice President, Senator Kashim She5ma -- Senate President, Dis7nguished Senator Godswill Akpabio -- Right Honourable Speaker, Tajudeen Abbas -- Na7onal Chairman of our great party, Excellency, Abdullahi Umar Ganduje -- State Governors here present -- Dis7nguished Leaders and Members of the Na7onal Assembly -- Senior Government Oﬃcials here present -- Gentlemen of the press, My Fellow Nig

In [ ]:
# Import the necessary libraries and vector store to create an in-memory search index for the documents 
from langchain_community.vectorstores import DocArrayInMemorySearch 

# assign the loaded pages to a variable and create an in-memory vector store using the embeddings model 
vectorstore = DocArrayInMemorySearch.from_documents(pages, embedding=embeddings)

In [ ]:
# Import the necessary retriever to search the vector store for relevant documents based on a query 
retriever = vectorstore.as_retriever()
retriever.invoke('Agenda')

[Document(metadata={'producer': 'macOS Version 10.14.6 (Build 18G9323) Quartz PDFContext', 'creator': 'Word', 'creationdate': "D:20241218144408Z00'00'", 'title': 'Microsoft Word - Document2', 'moddate': "D:20241218144408Z00'00'", 'keywords': '', 'aapl:keywords': '[]', 'source': '2025 Budget Performance.pdf', 'total_pages': 6, 'page': 2, 'page_label': '3'}, page_content='13. These clear results of gradual recovery, among others, reﬂect the resilience of our economy and the impact of deliberate policy choices we made from the outset.  2024 BUDGET PERFORMANCE  14. I am happy to inform this Na1onal Assembly that our administra1on abained remarkable milestones in implemen1ng the 2024 Budget. In 2024, we achieved: o 14.55 trillion naira in revenue, mee1ng 75 percent of our target as of the third quarter. o 21.60 trillion naira in expenditure, represen1ng 85 percent of our target, also in the third quarter.  15. While challenges persist, we improved revenue collec1on and fulﬁlled key obliga1o

In [ ]:
# Import the necessary libraries to create a chain that combines the retriever prompt model and parser to answer questions based on the context retrieved from the vector store 
from operator import itemgetter
chain = (
    {
        "context": itemgetter("question") | retriever,
        "question": itemgetter("question"),
    }
    | prompt 
    | model
    | parser
)

In [ ]:
# Define a list of questions to ask the model based on the context retrieved from the vector store 
questions = [
    "What is the budget for 2025?",
    "What are the main objectives of the budget?",
    "How much is allocated for education?",
    "What is the total budget amount?",
    "Are there any new initiatives in the budget?"
]
for question in questions:
    print(f"Question: {question}")
    print(f"Answer: {chain.invoke({'question': question})}")
    print()

Question: What is the budget for 2025?
Answer: The document mentions "the 2025 Budget Proposal" rather than a specific name of the budget. However, it does provide details about the key components and goals of the proposed 2025 budget.

To answer your question directly based on the context provided:

1. There is no explicit mention of a 2025 budget.
2. The document mentions "the 2025 Budget Proposal," suggesting that this could be the name or title given to the proposed budget.
3. However, there are hints that the 2025 Budget may refer specifically to the "Budget of Restoration: Securing Peace, Rebuilding Prosperity" mentioned in a section of the document.

In summary, I don't have enough information to accurately answer your question about what specific budget it refers to.

Question: What are the main objectives of the budget?
Answer: I don't know.

Question: How much is allocated for education?
Answer: According to the document, 826.90 billion naira is allocated for infrastructure d